### A bit more about decorators

## Running at import time

Key feature декораторов в том, что они применяются сразу после объявления декорируемой функции во время загрузки модуля в Python

In [1]:
registry = []

def register(func):
    print(f"register: {func}")
    registry.append(func)
    return func

@register
def f1():
    print("running f1")

@register
def f2():
    print("running f2")

def f3():
    print("running f3")

register: <function f1 at 0x7c6ee3f69240>
register: <function f2 at 0x7c6ee3f688b0>


In [2]:
f1()
f2()

running f1
running f2


In [3]:
f3()

running f3


In [4]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

## Декоратор-логгер

### logging

Модуль `logging` содержит набор функций для логирования различного поведения в вашей программе. Например, для понимания, что ваша программа работает как ожидается, или там идет что-то не так

Есть разные уровни логирования: debug, info, warning, error, critical

другая реализация логера: [loguru](https://loguru.readthedocs.io/en/stable/)

In [5]:
import logging

In [6]:
logging.info('Working as expected')
logging.warning('Smth might go wrong, check this!')

Можно логировать в файл (надо начать новую сессию в Python)

In [1]:
!true > example.log

In [2]:
import os
import logging

log_filename = os.path.join('./example.log')
log_level = logging.DEBUG

logging.basicConfig(
    filename=log_filename,
    encoding='utf-8',
    level=log_level,
    force=True,  # https://stackoverflow.com/questions/54597462/problem-with-logging-module-in-google-colab
    format='%(asctime)s %(message)s'
)

In [3]:
logging.debug("debug")
logging.info("info")
logging.warning("warning")
logging.error("error")

In [4]:
!cat example.log

2023-09-29 11:51:25,956 debug
2023-09-29 11:51:25,958 info
2023-09-29 11:51:25,958 warning
2023-09-29 11:51:25,958 error


(надо начать новую сессию в Python)

In [1]:
import logging
import sys

# logger = logging.getLogger(name='log')
# logger.setLevel(logging.DEBUG)
# logger.addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
from functools import partial

def logged(func=None,
           *,
           log_level=logging.INFO,
           log_name=None,
           log_message=None):
    log_name = log_name if log_name else func.__name__
    logger = logging.getLogger(log_name)

    if func is None:
        return partial(logged, log_level=log_level, log_name=log_name, log_message=log_message)

    log_message = log_message if log_message else func.__name__
    def wrapper(*args, **kwargs):
        logger.log(log_level, log_message)
        return func(*args, **kwargs)
    return wrapper

In [3]:
@logged(log_name='log')
def add(x, y):
    return x + y

In [4]:
add(1,2)

3

## Classes

Initialization

In [5]:
class A:
    def __init__(self, param1, param2, secret_param, super_secret_param):
        self.param1 = param1
        self.param2 = param2
        self._secret_param = secret_param
        self.__super_secret_param = super_secret_param

In [6]:
a = A(1,2,3,4)

In [11]:
a.param1, a.param2

(1, 2)

In [12]:
a._secret_param

3

In [13]:
a.__super_secret_param

AttributeError: ignored

In [15]:
print(dir(a)[:3])

['_A__super_secret_param', '__class__', '__delattr__']


In [17]:
a._A__super_secret_param

4

In [18]:
class A:
    def __new__(cls):
        return super().__new__(cls)

In [19]:
a = A()

In [23]:
class Singleton:
    def __new__(cls):
        if not hasattr(cls, "instance"):
            cls.instance = super().__new__(cls)
        return cls.instance

In [24]:
# class Singleton:
#     def __new__(cls):
#         try:
#             return cls.instance
#         except AttributeError:
#             cls.instance = super().__new__(cls)
#             return cls.instance

In [25]:
a = Singleton()
b = Singleton()

a is b

True

In [26]:
class A:
    def __init__(self, a):
        self.a = a

    def method(self):
        print(f"in method of an object with a={self.a}")

    @staticmethod
    def static_method():
        print(f"in static method of A object")

    @classmethod
    def class_method(cls):
        print(f"In a class method of {cls}")

In [27]:
a = A(1)

In [28]:
a.method()

in method of an object with a=1


In [29]:
a.static_method()

in static method of A object


In [30]:
A.static_method()

in static method of A object


In [32]:
a.class_method()

In a class method of <class '__main__.A'>


In [33]:
A.class_method()

In a class method of <class '__main__.A'>


In [34]:
class Person:
    def __init__(self, name):
        self.name = name

    @classmethod
    def create_person(cls, name):
        print(f"In a class method of {cls}")
        return cls(name)

In [35]:
Person.create_person("A")

In a class method of <class '__main__.Person'>


### Наследование

In [46]:
class A:
    def __init__(self, a):
        self.a = a

    def method1(self):
        print("IN A, method1")

    def method2(self):
        print("IN A, method2")

    def method3(self):
        print("IN A, method3")

In [47]:
class B(A):
    def __init__(self, a, b):
        super().__init__(a)
        self.b = b

    def method2(self):
        print("IN B, method2")

    def method3(self):
        print("IN B, method3")
        super().method3()

In [48]:
a = A(1)
b = B(1,2)

In [49]:
a.method1()
a.method2()
a.method3()

IN A, method1
IN A, method2
IN A, method3


In [50]:
b.method1()
b.method2()
b.method3()

IN A, method1
IN B, method2
IN B, method3
IN A, method3


## Обработка исключений

In [51]:
a = '1234'

try:
    a[0] = '10'
except:
    print('impossible to modify object')

impossible to modify object


In [53]:
a = '1234'

try:
    a[0] = '10'
except:
    print('impossible to modify object')
else:
    print("NOTHING BAD HAPPENED")
finally:
    print("FINALLY IS CALLED")

impossible to modify object
FINALLY IS CALLED


In [54]:
a = '1234'

try:
    print(a)
except:
    print('impossible to modify object')
else:
    print("NOTHING BAD HAPPENED")
finally:
    print("FINALLY IS CALLED")

1234
NOTHING BAD HAPPENED
FINALLY IS CALLED


In [55]:
a = '1234'

try:
    a[0] = '10'
except BaseException:
    print('impossible to modify object')

impossible to modify object


In [58]:
issubclass(TypeError, BaseException), issubclass(AttributeError, BaseException), issubclass(ValueError, BaseException)

(True, True, True)

цепочка наследований

In [60]:
TypeError.mro()

[TypeError, Exception, BaseException, object]

In [61]:
class CustomStringException(BaseException):
    pass

In [63]:
a = '1234'

try:
    a[0] = '10'
except ValueError:
    print("DIDNT HAPPEN")
except BaseException:
    print("HERE!")
except TypeError:
    print("HERE!!!!!!!!")

HERE!


In [66]:
a = '1234'

try:
    a[0] = '10'
except TypeError as e:
    raise CustomStringException(e)

CustomStringException: ignored

In [67]:
class A(Exception):
    pass

class B(A):
    pass

class C(B):
    pass

for cls in [C, A, B]:
    try:
        print(f'raised {cls.__name__}')
        raise cls()

    except A:
        print('A')

    except C:
        print('C')

    except B:
        print('B')



raised C
A
raised A
A
raised B
A


In [68]:
for cls in [C, A, B]:
    try:
        print(f'raised {cls.__name__}')
        raise cls()

    except B:
        print('B')

    except C:
        print('C')

    except A:
        print('A')



raised C
B
raised A
A
raised B
B


In [69]:
for cls in [C, A, B]:
    try:
        print(f'raised {cls.__name__}')
        raise cls()

    except C:
        print('C')

    except B:
        print('B')

    except A:
        print('A')



raised C
C
raised A
A
raised B
B
